In [1]:

def analyseImg(img):
    client = vision.ImageAnnotatorClient()
    # construct an iamge instance

    content = io.BytesIO()
    img.save(content, format='PNG')

    image = types.Image(content=content.getvalue())
    response = client.text_detection(image=image)  # returns TextAnnotation

    texts = response.text_annotations
    #print(full_text.bounding_poly.vertices)
    plt.figure(figsize=(12, 20))
    plt.imshow(Image.open(content))
    for text in texts:    
        vertices = [[vertex.x, vertex.y] for vertex in text.bounding_poly.vertices]
        vertices.append(vertices[0]) #repeat the first point to create a 'closed loop'

        xs, ys = zip(*vertices) #create lists of x and y values
        plt.plot(xs,ys) 
    pltContent = io.BytesIO()
    plt.show()
    plt.savefig(pltContent, format='PNG')
    
    print(time.time())
    return pltContent # if you need...

class PhotoBoothApp:
    def __init__(self, vs):
        # store the video stream object and output path, then initialize
        # the most recently read frame, thread for reading frames, and
        # the thread stop event
        self.vs = vs
        self.frame = None
        self.thread = None
        self.stopEvent = None
        # initialize the root window and image panel
        self.root = tk.Tk()
        self.panel = None
        # create a button, that when pressed, will take the current
        # frame and save it to file
        btn = tk.Button(self.root, text="Snapshot!",
            command=self.takeSnapshot)
        btn.pack(side="bottom", fill="both", expand="yes", padx=10,
            pady=10)
        # start a thread that constantly pools the video sensor for
        # the most recently read frame
        self.stopEvent = threading.Event()
        self.thread = threading.Thread(target=self.videoLoop, args=())
        self.thread.start()
        # set a callback to handle when the window is closed

        self.root.wm_title("OCR")
        self.root.wm_protocol("WM_DELETE_WINDOW", self.onClose)

    def videoLoop(self):
    # DISCLAIMER:
    # I'm not a GUI developer, nor do I even pretend to be. This
    # try/except statement is a pretty ugly hack to get around
    # a RunTime error that Tkinter throws due to threading
        try:
            # keep looping over frames until we are instructed to stop
            while not self.stopEvent.is_set():
                # grab the frame from the video stream and resize it to
                # have a maximum width of 300 pixels
                self.frame = self.vs.read()
                self.frame = imutils.resize(self.frame, width=300)

                # OpenCV represents images in BGR order; however PIL
                # represents images in RGB order, so we need to swap
                # the channels, then convert to PIL and ImageTk format
                image = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
                image = Image.fromarray(image)
                image = ImageTk.PhotoImage(image)

                # if the panel is not None, we need to initialize it
                if self.panel is None:
                    self.panel = tk.Label(image=image)
                    self.panel.image = image
                    self.panel.pack(side="left", padx=10, pady=10)

                # otherwise, simply update the panel
                else:
                    self.panel.configure(image=image)
                    self.panel.image = image
        except RuntimeError:
            print("[INFO] caught a RuntimeError")

    def takeSnapshot(self):
    # save the file
        
        image1 = Image.fromarray(self.frame.copy())
        imgPlot = Image.open(analyseImg(image1))
        open(imgPlot)
        imgtk = ImageTk.PhotoImage(image=imgPlot)

        self.panel = tk.Label(image=imgtk)
        self.panel.image = imgtk
        self.panel.pack()

        print("[INFO] saved")

    def onClose(self):
        # set the stop event, cleanup the camera, and allow the rest of
        # the quit process to continue
        print("[INFO] closing...")
        self.vs.stop()
        self.root.quit()

vs = VideoStream().start()

tkapp = PhotoBoothApp(vs)
tkapp.root.mainloop()

NameError: name 'VideoStream' is not defined